<a href="https://colab.research.google.com/github/sugarcane-mk/dataPreps_speech/blob/main/Notebooks/DataPrep_4_ASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pydub

# Segmentation



The audio files are segmented into individual sentences using `.lab` files, which contain the start and end timestamps for each sentence. These timestamps guide the extraction of specific segments from the audio files, creating individual files for each sentence.






In [ ]:
# !pip install pydub
from pydub import AudioSegment
import IPython.display as ipd
with open('/content/drive/Shareddrives/Audio_ processing/Segmented_data/Punitha_FPU/label/478-487/478-487(10).lab','r') as f:  #lab file
  lines = f.readlines()

lines=lines[1::2]

# lines=lines[16::2]
# lines=['66275000 85800000 61\n',
#  '104650000 122900057 62\n',
#  '137975000 151775000 63\n',
#  '161975000 175725000 64\n',
#  '187150000 207500000 65\n',
#  '216250000 235299943 66\n',
#  '248850000 265975000 67\n',
#  '278700000 298274943 68\n',
#  '320200000 342550000 69\n',
# '383950000 407500057 70\n']

print('No. of sentences : ', len(lines))

lines


No. of sentences :  10


['55050000 75925000 478\n',
 '85775000 96900000 479\n',
 '161175000 190600000 480\n',
 '200200000 210550000 481\n',
 '216150000 225150000 482\n',
 '236100000 260524943 483\n',
 '281250000 298500113 484\n',
 '309500000 335025057 485\n',
 '341700000 360300113 486\n',
 '366625000 384500000 487\n']

In [ ]:
from pydub import AudioSegment
import IPython.display as ipd
import os
import re

audio_file = '/content/drive/Shareddrives/Audio_ processing/Segmented_data/Punitha_FPU/label/478-487/478-487(10).wav'   # Audio files path

match = re.search(r"/Segmented_data/([^/]+)/label/([^/]+)/", audio_file)  # re search to extract speaker and sentence folder name

# Speaker and session information
speaker_folder = match.group(1)      # speaker folder name
sentence_folder = match.group(2)     # sentence folder name
speaker = "FPU"                      # speaker name
session = 10                          # session number


for line in lines:
  parts = line.strip().split()
  start_time = float(parts[0]) / 10000
  end_time = float(parts[1]) / 10000
  sentence_number = int(parts[2])

  audio = AudioSegment.from_file(audio_file)
  extracted_audio = audio[start_time:end_time]

  output_file = f"/content/drive/Shareddrives/Audio_ processing/Segmented_data/{speaker_folder}/speech/{sentence_folder}/{speaker}_{sentence_number}_{session}.wav"   # MSU_1_1.wav # Audio file path
  os.makedirs(os.path.dirname(output_file), exist_ok=True)
  extracted_audio.export(output_file, format="wav")
  # print(f'File saved to path {output_file}')


print(f"Process completed for speaker {speaker_folder} for sentence {sentence_folder}, and session {session}.")

Process completed for speaker Punitha_FPU for sentence 478-487, and session 10.


MSR
*   131-155(7).....142 missing
*   131-155(9).....152 missing
*   156-180(5).....161 162 missing
*   156-180(8).....159-162 164 176 missing
*   156-180(9).....160-162 168 176 missing
*   181-196-all....186 number issue

FPU
*  262-285(10).....280 Missing

In [ ]:
from pydub import AudioSegment
import IPython.display as ipd
import os

audio_file = '/content/drive/Shareddrives/Audio_ processing/Segmented_data/MSU/label/156-180/156-180(10).wav'

# Speaker and session information
speaker = "MSU"
session = 10


for line in lines:
  parts = line.strip().split()
  start_time = float(parts[0]) / 10000
  end_time = float(parts[1]) / 10000
  sentence_number = int(parts[2])

  audio = AudioSegment.from_file(audio_file)
  extracted_audio = audio[start_time:end_time]

  output_file = f"/content/drive/Shareddrives/Audio_ processing/Segmented_data/{speaker}/speech/156-180/{speaker}_{sentence_number}_{session}.wav"
  os.makedirs(os.path.dirname(output_file), exist_ok=True)
  extracted_audio.export(output_file, format="wav")


print("done")

done


# Clean & Split Transcripts

In [4]:
# prompt: i have sentence number and transcript in a txt file eg. 1. kadxawulai wandandgu. i need to clean the text and  create spk2utt.txt it should contain speaker_sentence number_repeatcount tabspace transcript each transcript should ne repeated 10 times

import re

def create_spk2utt(input_file, output_file):
  """
  Processes a text file containing sentence numbers and transcripts,
  cleans the transcripts, and creates a spk2utt.txt file.

  Args:
    input_file: Path to the input text file.
    output_file: Path to the output spk2utt.txt file.
  """
  try:
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
      for line in infile:
        parts = line.strip().split('.', 1)
        if len(parts) == 2:
          sentence_number = parts[0].strip()
          transcript = parts[1].strip()

          # Clean the transcript
          transcript = re.sub(r'[^\w\s]', '', transcript)  # Remove punctuation
          transcript = transcript.lower()  # Convert to lowercase

          # Write to spk2utt.txt with 10 repetitions
          for i in range(1,11):
            outfile.write(f"FPU_{sentence_number}_{i}\t{transcript}\n")                   # Modify Speaker ID
  except FileNotFoundError:
      print(f"Error: Input file '{input_file}' not found.")
  except Exception as e:
      print(f"An error occurred: {e}")


# Example usage (replace with your actual file paths)
input_file_path = '/content/drive/Shareddrives/Audio_ processing/Segmented_data/Punitha_FPU/transctipts_FPU.txt'          #Change this to the path of the input file
output_file_path = '/content/drive/Shareddrives/Audio_ processing/Segmented_data/Punitha_FPU/spk_2_utt.txt'               #Change this to the path of the output file
create_spk2utt(input_file_path, output_file_path)

print(f'Created spk_2_utt file at {output_file_path}')

Created spk_2_utt file at /content/drive/Shareddrives/Audio_ processing/Segmented_data/Punitha_FPU/spk_2_utt.txt


In [ ]:
# Creates a .txt with speaker\t transcript <------------------------------ spk_2_utt.txt ---------------------------------->

# output.txt

# MSR_1_1 transcript
# MSR_1_2 transcript
# ....
# MSR_180_10 transcript

import os
import re

def clean_sentence(sentence):
    """
    Removes all symbols from the sentence, keeping only alphanumeric characters and spaces.
    """
    return re.sub(r'[^a-zA-Z0-9\s]', '', sentence)

def generate_copies(input_file, output_folder):
    """
    Generates 10 copies of each sentence, saves them as separate text files,
    and removes symbols from the sentences.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Read the input file
    with open(input_file, 'r', encoding='utf-8') as file:
        sentences = file.readlines()

    # Process each sentence
    for sentence_number, sentence in enumerate(sentences, start=1):
        # Clean the sentence by removing symbols
        clean_text = clean_sentence(sentence.strip())

        # Generate 10 copies for each sentence
        for copy_number in range(1, 11):                                                  # <-------------- Loop to create 10 copies of each transcript -------------------->
            # Construct the filename
            filename = "spk_2_utt.txt"                                                    # <-------------- Spk_2_utt.txt File name of the text file -------------------->
            file_path = os.path.join(output_folder, filename)

            # Write the cleaned sentence to the file
            with open(file_path, 'a', encoding='utf-8') as output_file:
                output_file.write(f"MSR_{sentence_number}_{copy_number}\t{clean_text}\n") # <------------------ Enter the speaker ID --------------------->

    print(f"Files generated in the folder: {output_folder}")


# Input and output paths
input_file = "/content/drive/Shareddrives/Audio_ processing/transctipts_200.txt"          # <-------------- Text file containing transcripts --------------------> Replace with your input file path
output_folder = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR"  # <-------------- Path to store the spk2utt file --------------------> Replace with your desired output folder

# Run the function
generate_copies(input_file, output_folder)


Files generated in the folder: /content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR


## Split spk2utt

In [ ]:
# to split each sentence in a spk_2_utt.txt file into seperate text files the file name should be the first word of sentence

# text/
#       MSR_1_1.txt -> MSR_1_1  transcript
#       MSR_1_2.txt -> MSR_1_2  transcript
#       .....
#       MSR_180_10.txt -> MSR_180_10  transcript

import re
import os

def split_sentences_into_files(input_filepath, output_folder):
    """
    Splits sentences from a text file into separate files,
    using the first word of each sentence as the filename.
    """
    try:
        with open(input_filepath, 'r', encoding='utf-8') as infile:
            for line in infile:
                sentence = line.strip()
                if not sentence:  # Skip empty lines
                    continue

                words = sentence.split()
                if not words: # Skip lines with no words
                    continue

                filename = words[0]
                # Sanitize the filename to remove invalid characters
                filename = re.sub(r'[\\/*?:"<>|]', "", filename)

                filepath = os.path.join(output_folder, f"{filename}.txt")

                with open(filepath, 'w', encoding='utf-8') as outfile:
                    outfile.write(sentence)
        print(f"Sentences split into files in: {output_folder}")

    except FileNotFoundError:
        print(f"Error: Input file not found at {input_filepath}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
input_file = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/spk_2_utt.txt"     # <-------------- Path to spk_2 utt.txt files -------------------->
output_directory = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/prompts"     # <-------------- Path to store all the splitted txt files -------------------->

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

split_sentences_into_files(input_file, output_directory)

# text/
#       MSR_1_1.txt -> MSR_1_1  transcript
#       MSR_1_2.txt -> MSR_1_2  transcript
#       .....
#       MSR_180_10.txt -> MSR_180_10  transcript

import re
import os
…        print(f"An error occurred: {e}")

# Example usage:
input_file = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/spk_2_utt.txt"     # <-------------- Path to spk_2 utt.txt files -------------------->
output_directory = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/prompts"     # <-------------- Path to store all the splitted txt files -------------------->

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

split_sentences_into_files(input_file, output_directory)

Sentences split into files in: /content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/prompts


split transcripts.txt

In [ ]:
# Creates .txt files for each audio with transcript only

# text/
#   |-  MSU_1_1.txt -> transcript
#   |   ...
#   |_  MSU_180_10.txt -> transcript


import os
import re

def clean_sentence(sentence):
    """
    Removes all symbols from the sentence, keeping only alphanumeric characters and spaces.
    """
    return re.sub(r'[^a-zA-Z0-9\s]', '', sentence)

def generate_copies(input_file, output_folder):
    """
    Generates 10 copies of each sentence, saves them as separate text files,
    and removes symbols from the sentences.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Read the input file
    with open(input_file, 'r', encoding='utf-8') as file:
        sentences = file.readlines()

    # Process each sentence
    for sentence_number, sentence in enumerate(sentences, start=1):
        # Clean the sentence by removing symbols
        clean_text = clean_sentence(sentence.strip())

        # Generate 10 copies for each sentence
        for copy_number in range(1, 11):
            # Construct the filename
            filename = f"MSU_{sentence_number}_{copy_number}.txt"
            file_path = os.path.join(output_folder, filename)

            # Write the cleaned sentence to the file
            with open(file_path, 'w', encoding='utf-8') as output_file:
                output_file.write(clean_text)

    print(f"Files generated in the folder: {output_folder}")


# Input and output paths
input_file = "/content/transliteration1-197.csv"                                                  # Replace with your input file path <-------------- Text file containing transcripts -------------------->
output_folder = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/text"  # Replace with your desired output folder <-------------- Path to store all the splitted txt files -------------------->

# Run the function
generate_copies(input_file, output_folder)


Files generated in the folder: /content/drive/Shareddrives/Audio_ processing/Segmented_data/MSU/text


# Lexicon generation

### UniQue word Identification

In [ ]:
# Unique words

import re
import os

def process_text_files(input_filepath, output_filepath):
    """
    Processes text files to remove unwanted symbols, split into words,
    and store unique words in a new text file.
    """
    unique_words = set()

    try:
        with open(input_filepath, 'r', encoding='utf-8') as infile:
            for line in infile:
                # Remove unwanted symbols (keep alphanumeric and spaces)
                cleaned_line = re.sub(r'[^\w\s]', '', line).lower()  # Convert to lowercase

                # Split the line into words
                words = cleaned_line.split()

                # Add unique words to the set
                unique_words.update(words)

        # Write unique words to the output file
        with open(output_filepath, 'w', encoding='utf-8') as outfile:
            for word in sorted(unique_words):  # Sort for better readability
                outfile.write(word + '\n')

        print(f"Unique words saved to: {output_filepath}")

    except FileNotFoundError:
        print(f"Error: Input file not found at {input_filepath}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:  Replace with your actual file paths
input_file = "/content/drive/Shareddrives/Audio_ processing/transctipts_200.txt"                      # <------------------ Input file containing transcripts ----------------->
output_file = "/content/drive/Shareddrives/Audio_ processing/unique_words_200.txt"                    # <------------------ Output file to store unique words in transcripts --------------->

process_text_files(input_file, output_file)

Unique words saved to: /content/drive/Shareddrives/Audio_ processing/unique_words_200.txt


In [ ]:
# prompt: i have file 1 and file 2 .txt i need to compare  words in file 1 with the 1st word in file 2 and to display the unmatches words in file 1

def compare_words(file1_path, file2_path):
    """
    Compares words in file1 with the first word in file2 and displays unmatched words from file1.
    """
    try:
        with open(file1_path, 'r', encoding='utf-8') as file1, open(file2_path, 'r', encoding='utf-8') as file2:
            file1_words = set()
            for line in file1:
                words = line.strip().split()  # Split each line into words
                file1_words.update(words)

            first_word_file2 = file2.readline().strip().split()[0] # Get only the 1st word in file2

            unmatched_words = file1_words - set([first_word_file2]) #difference between file1 and file2

            for word in unmatched_words:
                print(word)

    except FileNotFoundError:
        print(f"Error: One or both of the files were not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


# Example usage: Replace with your actual file paths
file1_path = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/MSU/unique_words.txt"
file2_path = "/content/drive/MyDrive/lexicon.txt"
compare_words(file1_path, file2_path)

# Phone Dictonary

In [ ]:
# prompt: i have phone list and words  in a txt file i need to split words into phones by  phone list amd save it in a txt file as word \t phonems example aamaa aa m aa here is the phone list aa
# ai
# au
# dx
# ee
# eu
# ii
# lx
# nd
# ng
# nj
# nx
# rx
# sx
# tx
# oo
# uu
# zh
# e
# f
# g
# h
# i
# j
# k
# l
# m
# n
# o
# p
# r
# s
# t
# u
# w
# y
# a
# b
# c
# d

def split_words_into_phones(input_filepath, phone_list_filepath, output_filepath):
    """
    Splits words in a text file into phonemes based on a provided phone list.
    """
    try:
        with open(phone_list_filepath, 'r', encoding='utf-8') as phone_file:
            phone_list = [line.strip() for line in phone_file]

        with open(input_filepath, 'r', encoding='utf-8') as infile, open(output_filepath, 'w', encoding='utf-8') as outfile:
            for line in infile:
                word = line.strip()
                phonemes = []
                for phone in phone_list:  # Iterate through the phone list
                    if phone in word:
                        phonemes.append(phone)

                outfile.write(f"{word}\t{' '.join(phonemes)}\n")  # Write word and phonemes
        print(f"Word-phoneme mapping saved to: {output_filepath}")

    except FileNotFoundError:
        print(f"Error: Input file not found at {input_filepath} or {phone_list_filepath}")
    except Exception as e:
        print(f"An error occurred: {e}")


# Example usage:  Replace with your actual file paths
input_file = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/MSU/unique_words.txt"  # input file
phone_list_file = "/content/drive/MyDrive/lexicon.txt" # phone list file
output_file = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/MSU/word_phonemes.txt"  # output file

split_words_into_phones(input_file, phone_list_file, output_file)

Word-phoneme mapping saved to: /content/drive/Shareddrives/Audio_ processing/Segmented_data/MSU/word_phonemes.txt


In [ ]:
# Phonemes that should stay together
grouped_phonemes = [
    "aa", "ai", "au", "dx", "ee", "eu", "ii", "lx", "nd", "ng", "nj", "nx",
    "rx", "sx", "tx", "oo", "uu", "zh"
]

# Function to split words into phonemes
def split_into_phonemes(word, grouped_phonemes):
    phonemes = []
    i = 0
    while i < len(word):
        # Check for grouped phonemes
        matched = False
        for phoneme in grouped_phonemes:
            if word[i:i+len(phoneme)] == phoneme:
                phonemes.append(phoneme)
                i += len(phoneme)
                matched = True
                break
        # If no grouped phoneme matched, add the single character
        if not matched:
            phonemes.append(word[i])
            i += 1
    return phonemes

# Input and Output File Paths
input_file = "/content/drive/Shareddrives/Audio_ processing/unique_words_200.txt"                         # <-------------- File containing the words to be processed --------------->
output_file = "/content/drive/Shareddrives/Audio_ processing/phone_output_200.txt"                      # <-------------- File to save the phoneme sequences ----------------->

# Read words from the input file
with open(input_file, "r") as infile:
    words = [line.strip() for line in infile.readlines()]

# Generate phoneme sequences for each word
results = []
for word in words:
    phoneme_sequence = split_into_phonemes(word, grouped_phonemes)
    results.append(f"{word}\t{' '.join(phoneme_sequence)}")

# Write the results to the output file
with open(output_file, "w") as outfile:
    for line in results:
        outfile.write(line + "\n")

print(f"Phoneme sequences written to {output_file}")


Phoneme sequences written to /content/drive/Shareddrives/Audio_ processing/phone_output_200.txt


# Audio normalization

In [ ]:
import os
from pydub import AudioSegment
from pydub.effects import normalize

def normalize_audio_files(input_dir, output_dir):
    """
    Normalize audio files, including those in subfolders.

    Args:
        input_dir (str): Directory containing input audio files (including subfolders).
        output_dir (str): Directory to save normalized audio files, maintaining folder structure.
    """
    for root, dirs, files in os.walk(input_dir):  # Traverse all subdirectories
        for filename in files:
            if filename.endswith((".wav", ".mp3", ".flac", ".ogg")):  # Add other formats if needed
                input_path = os.path.join(root, filename)

                # Recreate the subfolder structure in the output directory
                relative_path = os.path.relpath(root, input_dir)
                output_subdir = os.path.join(output_dir, relative_path)
                if not os.path.exists(output_subdir):
                    os.makedirs(output_subdir)

                output_path = os.path.join(output_subdir, filename)

                try:
                    print(f"Processing: {input_path}")

                    # Load the audio file
                    audio = AudioSegment.from_file(input_path)

                    # Normalize the audio
                    normalized_audio = normalize(audio)

                    # Export the normalized audio to the output directory
                    normalized_audio.export(output_path, format="wav")  # Ensure output is in `.wav` format
                    print(f"Saved normalized file to: {output_path}")

                except Exception as e:
                    print(f"Error processing {input_path}: {e}")

# Set input and output directories
input_directory = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/speech"
output_directory = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/speech_normalised"

# Run the normalization
normalize_audio_files(input_directory, output_directory)


Processing: /content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/speech/1-20/MSU_1_1.wav
Saved normalized file to: /content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/speech_normalised/1-20/MSU_1_1.wav
Processing: /content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/speech/1-20/MSU_2_1.wav
Saved normalized file to: /content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/speech_normalised/1-20/MSU_2_1.wav
Processing: /content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/speech/1-20/MSU_3_1.wav
Saved normalized file to: /content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/speech_normalised/1-20/MSU_3_1.wav
Processing: /content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/speech/1-20/MSU_4_1.wav
Saved normalized file to: /content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/speech_normalised/1-20/MSU_4_1.wav
Processing: /content/drive/S

In [ ]:
# prompt: i have files in sub dirs such as MSU_1_1.wav, MSU_2_1.wav so on i need to rename it as MSR_1_1.wav , MSR_2_1.wav..

import os
import re

def rename_files(root_dir):
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            if file.startswith("MSU_") and file.endswith(".wav"):
                old_path = os.path.join(subdir, file)
                new_file = file.replace("MSU", "MSR", 1)
                new_path = os.path.join(subdir, new_file)
                os.rename(old_path, new_path)
                # print(f"Renamed '{old_path}' to '{new_path}'")
        print("All files renamed successfully.")

# Example usage (replace with your actual directory)
root_directory = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/speech_normalised/"
rename_files(root_directory)

All files renamed successfully.
All files renamed successfully.
All files renamed successfully.
All files renamed successfully.
All files renamed successfully.
All files renamed successfully.
All files renamed successfully.
All files renamed successfully.
All files renamed successfully.


# MISC

In [ ]:
with open('/content/41-60(1).lab','r') as f:
  lines = f.readlines()

lines[1::2]

['45800000 73875000 41\n',
 '84300000 99725000 42\n',
 '119475000 152550000 43\n',
 '188000000 227850000 44\n',
 '256400000 284550000 45\n',
 '342125000 385500000 46\n',
 '438900000 472625000 47\n',
 '698350000 746800000 48\n',
 '806025000 823900000 49\n',
 '877125000 877150000 50\n',
 '914375000 943100000 51\n',
 '1080575000 1121275000 53\n',
 '1163100000 1203050000 54\n',
 '1250200000 1293700000 55\n',
 '1346825000 1385125000 56\n',
 '1415900000 1445600000 57\n',
 '1605175000 1638225000 59\n']

In [ ]:
# prompt: given start and end times, use pydub for audio segmentation. The resulting wav should be in .wav

from pydub import AudioSegment

# Replace with your actual file paths and desired start and end times
audio_file = "/content/41-60(1).wav"

count= 0

for line in lines[1::2]:
  count+=1
  start_time, end_time = line.split(' ')[0:2]
  start_time_ms = float(start_time)/10000  # Start time in milliseconds
  end_time_ms = float(end_time)/10000    # End time in milliseconds

  audio = AudioSegment.from_file(audio_file)
  extracted_audio = audio[start_time_ms:end_time_ms]

  output_file = "FGA"+str(count)+"_1.wav"
  extracted_audio.export(output_file, format="wav")

In [ ]:
import os

# Replace with your actual file paths and desired start and end times
audio_file = "/content/drive/Shareddrives/NIEPMD DATA COLLECTION/Data collection _ Annotation/Data/Suradha(MSU)/Speech/1-20/1-20(1).wav"

count= 1

for line in lines[1::2]:
  start_time, end_time = line.split(' ')[0:2]
  start_time_ms = float(start_time)/10000  # Start time in milliseconds
  end_time_ms = float(end_time)/10000    # End time in milliseconds

  audio = AudioSegment.from_file(audio_file)
  extracted_audio = audio[start_time_ms:end_time_ms]

  output_file = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/MSU/MSU_"+str(count)+"_1.wav"
  os.makedirs(os.path.dirname(output_file), exist_ok=True)
  extracted_audio.export(output_file, format="wav")
  count+=1
print("done")


done


In [ ]:
import IPython.display as ipd

ipd.Audio('/content/drive/Shareddrives/Audio_ processing/Segmented_data/FGA/FGA41_1.wav')

In [ ]:
# prompt: how do we store the extracted audio files in drive
# !pip install pydub
from pydub import AudioSegment
import IPython.display as ipd
with open('/content/drive/Shareddrives/Audio_ processing/Segmented_data/MSU/label/131-155/131-155(4).lab','r') as f:
  lines = f.readlines()

lines=lines[1::2]

# lines=lines[16::2]



print('no of sentences : ', len(lines))

lines


no of sentences :  25


['31375000 54100000 131\n',
 '55575000 77050000 132\n',
 '77900000 101950000 133\n',
 '102250000 126675000 134\n',
 '127775000 165675000 135\n',
 '167200000 195450000 136\n',
 '206600000 220825000 137\n',
 '222025000 235575000 138\n',
 '254750000 275950000 139\n',
 '277050000 298175000 140\n',
 '298950000 313600000 141\n',
 '314825000 335325000 142\n',
 '337275000 369275000 143\n',
 '391000000 417875000 144\n',
 '419575000 437850000 145\n',
 '438825000 472100000 146\n',
 '501025000 519550000 147\n',
 '520925000 541950000 148\n',
 '543250000 557150000 150\n',
 '573075000 587450000 149\n',
 '605025000 630750000 151\n',
 '631900000 655050000 152\n',
 '697225000 732725000 153\n',
 '733875000 777825000 154\n',
 '795600000 824500000 155\n']

In [ ]:
# prompt: /content/drive/Shareddrives/Audio_ processing/Segmented_data/Punitha_FPU/label/131-155/131-155(1).wav i need to exeact the speaker name Punitha FPU and folder 131-155

import re

def extract_info(filepath):
  """
  Extracts speaker name and folder number from a given filepath.

  Args:
    filepath: The path to the audio file.

  Returns:
    A tuple containing the speaker name and folder number, or None if
    the information cannot be extracted.
  """
  match = re.search(r"/Segmented_data/([^/]+)/label/([^/]+)/", filepath)
  if match:
    speaker_name = match.group(1)
    folder_number = match.group(2)
    return speaker_name, folder_number
  else:
    return None

filepath = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Punitha_FPU/label/131-155/131-155(1).wav"
info = extract_info(filepath)

if info:
  speaker_name, folder_number = info
  print(f"Speaker Name: {speaker_name}")
  print(f"Folder Number: {folder_number}")
else:
  print("Could not extract speaker name and folder number from the filepath.")